# AISE 26 – W9D1 Split Strategy Showdown  
### Partner A: Random Holdout + 5-Fold Standard CV  
**Author:** Andrea Churchwell  
**Dataset:** Diabetes Progression (#7)  
**Task Type:** Regression  
**Metric:** R² (can be changed later if needed)  

---

## 📌 Notebook Purpose

This notebook helps me (Partner A) understand **every single step** of my evaluation strategy **before** I convert the final version into the required `eval_partner_a.py` script for the assignment.

I am working slowly and clearly to make sure:
- I follow the instructions *exactly*
- I understand each line of code
- I know why we split the data the way we do
- I know how R² is calculated
- I know what K-Fold CV does and why it’s required

This notebook is my learning space.  
The clean Python file (`eval_partner_a.py`) will be produced **after** I understand everything here.

---

## ✔ Partner A Requirements (What I Must Do)

- Perform an **80/20 Random Holdout split**
- Use the **Ridge** regression model (no tuning)
- Use the **same metric** as my partner (R² for now)
- Run **5-Fold Standard KFold Cross-Validation** on the **training** set only
- Print:
  - Test score (R²)
  - CV mean score
  - CV standard deviation
  - Individual fold scores

---

## ✔ What This Notebook Will Include

1. Setting up imports  
2. Loading the Diabetes dataset  
3. Inspecting the data  
4. Creating the 80/20 split  
5. Building the pipeline (StandardScaler + Ridge)  
6. Fitting the model  
7. Evaluating using R²  
8. Running 5-Fold KFold CV  
9. Understanding the outputs  
10. Preparing results for the required `comparison.csv`

---

## 🧠 Reminder

This notebook is for learning and exploring.  
Only after completing everything here, I will move the final version into:

`eval_partner_a.py`
to submit on GitHub as required.

In [157]:
import sys
sys.executable

'c:\\Users\\achur\\desktop\\AISE_CLASS_CLONES\\aise-w9d1-splitstrategy-churchwell-diaz\\venv\\Scripts\\python.exe'

In [158]:
import numpy as np            # numerical arrays and math
import pandas as pd           # table-style dataframes

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import plotly.express as px 
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
RANDOM_STATE

42

In [159]:
data = load_diabetes(as_frame=True)

X = data.data
y = data.target

print("Shape of X (features):", X.shape)
print("Shape of y (target):", y.shape)

X.head()

Shape of X (features): (442, 10)
Shape of y (target): (442,)


,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641


In [160]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=RANDOM_STATE
)

print("X_train shape:", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape: ", y_test.shape)

X_train shape: (353, 10)
X_test shape:  (89, 10)
y_train shape: (353,)
y_test shape:  (89,)


In [161]:
# Pipeline: StandardScaler + Ridge
model = Pipeline([
    ("scaler", StandardScaler()),
    ("ridge", Ridge())
])

# Fit on training data
model.fit(X_train, y_train)

# --- Training set R² ---
y_pred_train = model.predict(X_train)
train_r2 = r2_score(y_train, y_pred_train)
print(f"Train R² score: {train_r2:.4f}")

# --- Test set R² (20% holdout) ---
y_pred_test = model.predict(X_test)
test_r2 = r2_score(y_test, y_pred_test)
print(f"Test R² score (20% holdout): {test_r2:.4f}")

Train R² score: 0.5276
Test R² score (20% holdout): 0.4541


In [162]:
kfold = KFold(
    n_splits=5,
    shuffle=True,
    random_state=RANDOM_STATE
)

cv_scores = cross_val_score(
    model,
    X_train,
    y_train,
    cv=kfold,
    scoring="r2"
)

print("Individual fold scores:", cv_scores)
print(f"CV Mean R²: {cv_scores.mean():.4f}")
print(f"CV Std Dev: {cv_scores.std():.4f}")

Individual fold scores: [0.46995096 0.53808555 0.41334996 0.4902986  0.49223599]
CV Mean R²: 0.4808
CV Std Dev: 0.0404


In [163]:
summary_df = pd.DataFrame({
    "metric": ["Train R²", "Test R²", "CV Mean R²", "CV Std R²"],
    "value": [
        train_r2,
        test_r2,
        cv_scores.mean(),
        cv_scores.std()
    ]
})
summary_df


,metric,value
0,Train R²,0.527632
1,Test R²,0.454147
2,CV Mean R²,0.480784
3,CV Std R²,0.040422


### 📌 5-Fold Cross-Validation (R²)

This chart shows the R² score for each of the 5 folds during cross-validation on the **training set only**.

Cross-validation helps us answer:
- **How stable is the model?**
- **Does performance change a lot depending on which data it sees?**
- **Is the model overfitting or generalizing well?**

In this case, the scores across the 5 folds are relatively close together, which means:
- The model behaves consistently  
- There is no major variance between folds  
- Our Ridge regression model is fairly stable on this dataset

In [164]:
cv_df = pd.DataFrame({
    "Fold": ["Fold 1", "Fold 2", "Fold 3", "Fold 4", "Fold 5"],
    "R² Score": cv_scores
})

fig = px.bar(
    cv_df,
    x="Fold",
    y="R² Score",
    title="📊 5-Fold Cross-Validation R² Scores (Partner A – Random Holdout)",
    text="R² Score",
    color="R² Score",
    color_continuous_scale="Bluered" 
)

fig.update_layout(
    title_font_size=22,
    xaxis_title="Fold Number",
    yaxis_title="R² Score",
    height=500,
    width=900,
    template="plotly_white"
)

fig.update_traces(
    texttemplate='%{text:.3f}',
    textposition='outside'
)

fig.show()

### 🎯 Actual vs Predicted (Test Set)

This scatter plot compares the **true target values** vs. the **model's predictions** on the 20% test set.

What this chart tells us:
- Each point represents a single patient in the test dataset.
- The dashed red diagonal line represents **perfect predictions**.
- Points close to the line = accurate predictions  
- Points far from the line = prediction error

In our model:
- The pattern is fairly linear, showing the model captured real signal.
- But the spread around the diagonal indicates some prediction error, which is expected because this is a noisy medical dataset.

This visualization helps us understand the model’s real-world performance beyond just the R² number.


In [165]:
# Build a DataFrame for actual vs predicted on the test set
test_results = pd.DataFrame({
    "Actual": y_test.reset_index(drop=True),
    "Predicted": pd.Series(y_pred_test).reset_index(drop=True)
})

fig = px.scatter(
    test_results,
    x="Actual",
    y="Predicted",
    title="🎯 Actual vs Predicted Values (Test Set – Ridge Regression)",
    labels={"Actual": "Actual Progression", "Predicted": "Predicted Progression"},
    opacity=0.7
)

# Make the chart bigger and cleaner
fig.update_layout(
    title_font_size=22,
    height=600,
    width=800,
    template="plotly_white"
)

# Add a perfect-prediction diagonal line
fig.add_shape(
    type="line",
    x0=test_results["Actual"].min(),
    y0=test_results["Actual"].min(),
    x1=test_results["Actual"].max(),
    y1=test_results["Actual"].max(),
    line=dict(color="red", dash="dash")
)

fig.show()

### 📉 Residual Plot – Test Set

This plot shows how far each prediction is from the actual value (Predicted − Actual).  
A good model has residuals scattered randomly around **0** with no clear pattern.

- Points above 0 → the model overpredicted  
- Points below 0 → the model underpredicted  
- A tight cluster around 0 → good accuracy  

In [166]:
residuals = y_pred_test - y_test.reset_index(drop=True)

fig = px.scatter(
    x=y_pred_test,
    y=residuals,
    title="📉 Residual Plot – Test Set",
    labels={"x": "Predicted Values", "y": "Residuals (Pred - Actual)"},
    opacity=0.7
)

fig.add_shape(
    type="line",
    x0=min(y_pred_test),
    y0=0,
    x1=max(y_pred_test),
    y1=0,
    line=dict(color="red", dash="dash")
)

fig.update_layout(template="plotly_white", height=500, width=800)
fig.show()

### 📊 Residual Distribution

This histogram shows how prediction errors are distributed.  
A symmetric distribution centered around 0 means the model is **not biased** toward overpredicting or underpredicting.


In [167]:
fig = px.histogram(
    residuals,
    nbins=30,
    title="📊 Distribution of Residuals",
    labels={"value": "Residual"},
    opacity=0.75
)

fig.update_layout(template="plotly_white", height=500, width=800)
fig.show()

### 📈 Train vs Test vs CV Mean R²

This bar chart compares three different evaluation metrics:
- **Train R²:** How well the model fits the training data  
- **Test R²:** How well the model generalizes to unseen data  
- **CV Mean R²:** Model stability across 5 different folds  

A good model has:
- Train R² slightly higher than Test R²  
- CV Mean R² sitting in between (which ours does)

In [168]:
compare_df = pd.DataFrame({
    "Metric": ["Train R²", "Test R²", "CV Mean R²"],
    "Score": [train_r2, test_r2, cv_scores.mean()]
})

fig = px.bar(
    compare_df,
    x="Metric",
    y="Score",
    title="📈 Train vs Test vs CV Mean R²",
    text="Score",
    color="Metric",
    color_discrete_sequence=px.colors.qualitative.Set2
)

fig.update_traces(texttemplate='%{text:.3f}', textposition='outside')
fig.update_layout(template="plotly_white", height=500, width=800)
fig.show()

### 📏 Actual vs Predicted (Sorted Test Set)

In this visualization, the test samples are sorted by their true target values (actual disease progression).  
Sorting them creates a smooth increasing line, which allows us to visually compare the **shape** of the model’s predictions to the true underlying pattern.

- The **Actual** line shows the true progression scores  
- The **Predicted** line shows the model’s output for each of those same samples  
- If the model is performing well, the predicted line should follow the same general trajectory as the actual line

This plot helps answer two important questions:

1. **Is the model capturing the overall trend?**  
   Yes — the predicted line follows the shape of the actual values, which indicates the model understands the general relationship between features and progression score.

2. **Where does the model struggle?**  
   The areas where the two lines diverge (larger gaps) indicate test samples that are harder for the model to predict accurately.

Even though the model is not perfect, this chart shows that it successfully captures the overall increasing pattern of the target values, which supports the R² results from the train/test split and cross-validation.


In [169]:
# Clean Series for actual + predicted
y_test_series = y_test.reset_index(drop=True)
y_pred_series = pd.Series(y_pred_test).reset_index(drop=True)

# Get sorted indices based on the actual values
sorted_idx = y_test_series.argsort()

# Sort both actual & predicted using the same index
actual_sorted = y_test_series.iloc[sorted_idx].reset_index(drop=True)
pred_sorted = y_pred_series.iloc[sorted_idx].reset_index(drop=True)

# X-axis: just positions 0..n-1 as a LIST (not range)
x_vals = list(range(len(actual_sorted)))

# Base line for Actual
fig = px.line(
    x=x_vals,
    y=actual_sorted,
    labels={"x": "Sorted Samples", "y": "Progression Score"},
    title="📏 Actual vs Predicted (Sorted Test Set)"
)

# Add Predicted line
fig.add_scatter(
    x=x_vals,
    y=pred_sorted,
    mode="lines",
    name="Predicted Values"
)

fig.update_layout(
    template="plotly_white",
    height=600,
    width=900
)

fig.show()
